In [12]:
import numpy as np
import matplotlib.pyplot as plt
from emitter import Particle, Emitter

In [13]:
cmin = 0
cmax = 2
n_points = 40

In [14]:
c = np.linspace(cmin, cmax, n_points)

In [15]:
f = lambda x: -x**2 + 2.1 * x - .3
y = f(c)
y = [max(0, _) for _ in y]

In [16]:
h = np.ones((n_points, 1))
h_vel = np.zeros_like(h)
for i in range(n_points // 2):
    h[i] = 3

In [17]:
def h_update(h, h_vel, y, damping=.99, c2=1, h2=2, dt=.45):
    h_new = np.copy(h)
    h_vel_new = np.copy(h_vel)
    for i in range(n_points):
        left_i = max(0, i - 1)
        right_i = min(n_points - 1, i + 1)
        Vleft, Vright = 0, 0
        if h[left_i] > 0:
            if (h[left_i] + y[left_i]) > (h[i] + y[i]):
                Vleft = min((h[left_i] + y[left_i]) - (h[i] + y[i]), h[left_i])
        if h[right_i] > 0:
            if (h[right_i] + y[right_i]) > (h[i] + y[i]):
                Vright = min((h[right_i] + y[right_i]) - (h[i] + y[i]), h[right_i])
        Vcur = h[i]
        h_new[left_i] -= Vleft * dt
        h_new[i] += Vleft * dt
        h_new[right_i] -= Vright * dt
        h_new[i] += Vright * dt
    return h_new, h_vel_new

In [18]:
def show_field(h, h_vel):
    for _, hv in zip(h, h_vel):
        print(f"{_} [{hv}]")

In [19]:
import bqplot
from bqplot import *
from bqplot import pyplot as plt
import ipywidgets as widgets

In [20]:
h = np.zeros((n_points, 1))
h_vel = np.zeros_like(h)
# for i in range(n_points // 2):
#     h[i] = .1

In [21]:
x_ord = OrdinalScale()
y_sc = LinearScale(min=0, max=4)

# bar = Bars(x=c, y=h[:, 0], scales={'x': x_ord, 'y': y_sc}, padding=0, stroke='steelblue', type='stacked')
bars = Bars(x=c, y=np.vstack([y, h[:, 0]]), scales={'x': x_ord, 'y': y_sc}, colors=["yellow", "steelblue"], type='stacked')
ax_x = Axis(scale=x_ord, visible=False)
ax_y = Axis(scale=y_sc, visible=True, orientation='vertical')
fig = Figure(marks=[bars], axes=[ax_x, ax_y], padding_x=0.025, padding_y=0.025, animation_duration=100)


def on_value_change(change):
    global h, h_vel, y, c
    with out:
        t = change['new']
        if t == 1:
            h = np.zeros((n_points, 1))
            h_vel = np.zeros_like(h)
            y = f(c)
            y = [max(0, _) for _ in y]
    #         for i in range(n_points // 2):
    #             h[i] = .1
        if t > 1:
            h, h_vel = h_update(h, h_vel, y)
            bars.y = np.vstack([bars.y[0], np.copy(h[:, 0])])
#             bar.x = c
        if t % 50 == 0 and t < 700:
#             with out:
            # emit particle on the left side
                h[1] += 1

slider = widgets.IntSlider(min=0, max=1000, step=1, continuos_update=True)
play = widgets.Play(min=1, max=1000, interval=30)
out = widgets.Output()
slider.observe(on_value_change, 'value')
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.VBox([widgets.HBox([play, slider]), fig, out])

In [22]:
bar_surf = Bars(x=c, y=y, scales={'x': x_ord, 'y': y_sc}, colors=["yellow"])
fig = Figure(marks=[bar_surf], axes=[ax_x, ax_y])
fig

Figure(axes=[Axis(scale=OrdinalScale(), visible=False), Axis(orientation='vertical', scale=LinearScale(max=4.0…

In [23]:
import sys
sys.float_info.epsilon

2.220446049250313e-16